In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install calmap

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats
import math
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from prettytable import PrettyTable
from tqdm import tqdm
from statsmodels.tsa.stattools import adfuller
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
train = pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/train.csv')
test = pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/test.csv')
train_calendar = pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/train_calendar.csv')
test_calendar = pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/test_calendar.csv')
submission = pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/solution_example.csv')

In [ ]:
def conv_datetime(dfs, date_column = 'date'):
    for df in dfs:
        df[date_column] = pd.to_datetime(df[date_column])

In [ ]:
conv_datetime([train,test,train_calendar,test_calendar])
train.drop(columns = ['id'], inplace=True)

This notebook aims to analyze the data provided for the **Rohlik Orders Forecasting Challenge**. The task is to predict the number of orders at selected warehouses for the next 60 days. The models will be evaluated on MAPE (mean absolute percentage erro) between predicted and actual orders. 

Based in Prague, the Rohlik Group is a European technology leader in e-grocery, with presence in several cities. Below, is a brief timeline of the company:
* **2015**: Expansion to Brno
* **2019**: Expansion to Budapest
* **2020**: Expansion to Austria
* **2021**: Expansion to Munich
* **2022**: Expansion to Frankfurt

**Warehouse available for the competition**: Prague_1, Prague_2, Prague_3, Brno_1, Frankfurt_1, Munich_1, Budapest_1 

## 1. INTRODUCTION

We have 5 files available:

* **train**: historical orders and selected features for each warehouse. The data is in daily frequence.
* **test**: "future" data that will be forecasted.
* **train_calendar**: a calendar for each warehouse for the train period.
* **test_calendar**: a calendar for each warehouse for the test period.
* **solution_example**: the submission format for this competition.

In [ ]:
train.head(3)

In [ ]:
test.head(3)

In [ ]:
print(f"Train data has {train.shape[0]} rows and {train.shape[1]} columns.")
print(f"Test data has {test.shape[0]} rows and {test.shape[1]} columns.")

In [ ]:
#missing values
train.isnull().sum() / len(train) * 100

Notes:

* Not all features in train set are present in test set.
* holiday_name is NaN when a given day is not a holiday or when holiday name was not registered. Holiday_name = NaN does not gurantee that a given day is not a holiday.
* Precipitation, snow and user activity could be forecasted alongside the target orders. 

In [ ]:
train['year'] = train['date'].dt.year
train['weekday'] = train['date'].dt.weekday
train['dayofyear'] = train['date'].dt.dayofyear
train['daynum'] = (train.date - train.date.iloc[0]).dt.days
train['weeknum'] = train['daynum'] // 7
train['month'] = train['date'].dt.to_period('M')
train['month_n'] = train['date'].dt.month
train['day'] = train['date'].dt.day

## 1.2 Range of Time Series

In [ ]:
row_counts_df = train.groupby(['warehouse']).size().reset_index(name='Entries')
dates_df = train.groupby(['warehouse'])['date'].agg(['min','max']).reset_index()
dates_df.columns = ['warehouse','Earliest_entry','Latest_entry']
train_summ = pd.merge(row_counts_df,dates_df, on = ['warehouse'])
train_summ

In [ ]:
row_counts_df_test = test.groupby(['warehouse']).size().reset_index(name='Entries')
dates_df_test = test.groupby(['warehouse'])['date'].agg(['min','max']).reset_index()
dates_df_test.columns = ['warehouse','Earliest_entry','Latest_entry']
test_summ = pd.merge(row_counts_df_test,dates_df_test, on = ['warehouse'])
test_summ

In [ ]:
def check_consecutive_rows(df):
    
    res = {}
    
    for warehouse in df['warehouse'].unique():
        warehouse_df = df[df['warehouse']==warehouse].sort_values('date')
        comp_range = pd.date_range(start=warehouse_df['date'].min(), end=warehouse_df['date'].max())
        actual = warehouse_df['date'].tolist()
        missing = [date for date in comp_range if date not in actual]
        
        res[warehouse] = {
            'is_consecutive': len(missing)==0,
            'missing_dates': missing,
            'missing_count': len(missing)
        }
        
    return res

results = check_consecutive_rows(train)

for warehouse,result in results.items():
    print(f"Warehouse: {warehouse}")
    if(result['is_consecutive']):
        print(f"  Status: Consecutive")
    else:
        print(f"  Status: Not Consecutive")
        print(f"  Total missing days: {result['missing_count']}")
        print("\n")

### 1.3 Common features

In [ ]:
group_df = train.groupby(['warehouse', 'month_n'])[['holiday','shops_closed','winter_school_holidays','school_holidays']].sum().reset_index()
warehouses = train['warehouse'].unique()

fig, axs = plt.subplots(nrows=(len(warehouses) // 3)+1,ncols=3,figsize=(16,12))
axs = axs.flatten()

for i, warehouse in enumerate(warehouses):
    data = group_df[group_df['warehouse'] == warehouse].set_index('month_n')
    data = data[['holiday','shops_closed','winter_school_holidays','school_holidays']]
    data.plot(kind='bar', stacked=True,ax=axs[i])
    
    axs[i].set_title(f"{warehouse}", fontsize=18)
    axs[i].set_xlabel('')
    axs[i].set_ylabel('')
    axs[i].legend(fontsize=14)
    axs[i].tick_params(axis='x', rotation=0)
    
for j in range(i+1, len(axs)):
    fig.delaxes(axs[j])

fig.tight_layout()
plt.show()

## 2. Time Series Overview

In [ ]:
plt.figure(figsize=(14,8))

for i, warehouse in enumerate(warehouses):
    df_warehouses = train[train['warehouse'] == warehouse]
    sns.lineplot(x='date',y='orders',data=df_warehouses, label=warehouse)

plt.title('Orders Over Time', fontsize=18)
plt.legend(fontsize=16)
plt.xticks(rotation=0)
plt.show()

Clear seasonal patterns for every warehouse.

* Prague, Brno, and Budapest seem to have a yearly seasonal pattern, with the peak in late December. These operations were all well-established when the historical data started being recorded, which is probably why we see a clear pattern for these locations.


In [ ]:
train.sort_values(['warehouse','date'], inplace=True)

for warehouse in warehouses:
    df_warehouse = train[train['warehouse']==warehouse]
    plt.figure(figsize=(12,5))
    plt.plot(df_warehouse['date'],df_warehouse['orders'],marker='',linestyle='-')
    plt.title(f"Orders over time for {warehouse}", fontsize=16)
    plt.xticks(rotation=45)
    
    ax=plt.gca()
    
        # Highlight school holiday periods
    school_holiday_periods = df_warehouse['school_holidays'] == 1
    school_holiday_start_end = (school_holiday_periods != school_holiday_periods.shift()).cumsum()
    
    for period in school_holiday_start_end.unique():
        if school_holiday_periods[school_holiday_start_end == period].any():
            school_holiday_period = df_warehouse[school_holiday_start_end == period]
            start = school_holiday_period['date'].min()
            end = school_holiday_period['date'].max()
            ax.axvspan(start, end, color='red', alpha=0.2)
    
    # Highlight winter school holiday periods
    winter_holiday_periods = df_warehouse['winter_school_holidays'] == 1
    winter_holiday_start_end = (winter_holiday_periods != winter_holiday_periods.shift()).cumsum()
    
    for period in winter_holiday_start_end.unique():
        if winter_holiday_periods[winter_holiday_start_end == period].any():
            winter_holiday_period = df_warehouse[winter_holiday_start_end == period]
            winter_start = winter_holiday_period['date'].min()
            winter_end = winter_holiday_period['date'].max()
            ax.axvspan(winter_start, winter_end, color='green', alpha=0.2)

plt.show()

### 2.1 Moving Average

In [ ]:
for warehouse in warehouses:
    df_warehouse = train[train['warehouse'] == warehouse]
    df_warehouse = df_warehouse.copy()

    plt.figure(figsize=(12, 5))
    
    # original time series
    plt.plot(df_warehouse['date'], df_warehouse['orders'], marker='', linestyle='-', label='Original', alpha=0.5)
    
    # 7-day moving average
    df_warehouse.loc[:,'7_day_ma'] = df_warehouse['orders'].rolling(window=7).mean()
    plt.plot(df_warehouse['date'], df_warehouse['7_day_ma'], marker='', linestyle='-', color='red', label='7 Day Moving Average')
    
    # 30-day moving average
    df_warehouse.loc[:,'30_day_ma'] = df_warehouse['orders'].rolling(window=30).mean()
    plt.plot(df_warehouse['date'], df_warehouse['30_day_ma'], marker='', linestyle='-', color='green', label='30 Day Moving Average')
        
    plt.title(f'Orders over Time for {warehouse}', fontsize=16)
    plt.xticks(rotation=0)
    plt.legend(fontsize=14)

    
plt.tight_layout()
plt.show()


### 2.2 Weekdays

In [ ]:
selected_warehouses = ['Prague_1', 'Prague_2', 'Prague_3', 'Brno_1', 'Budapest_1',
                      'Frankfurt_1', 'Munich_1']

# Group by week number, warehouse, and weekday
orders_per_week_warehouse_weekday = train.groupby(['weeknum', 'warehouse', 'weekday'])['orders'].sum().reset_index().pivot(index=['weeknum', 'warehouse'], columns='weekday')

# Fill NaN values with 0
orders_per_week_warehouse_weekday = orders_per_week_warehouse_weekday.fillna(0)

# Calculate the ratio of orders per weekday for each week and warehouse
ratio_orders_per_week_warehouse_weekday = orders_per_week_warehouse_weekday.apply(lambda row: row/sum(row) if sum(row) != 0 else row, axis=1).reset_index()

# Initialize an empty DataFrame to store median ratios
ratio_weekday = pd.DataFrame(columns=selected_warehouses, index=range(7), data=[[0, ]*len(selected_warehouses)]*7)

# Create subplots for each selected warehouse
fig, ax = plt.subplots(nrows=len(selected_warehouses), figsize=(24, 5*len(selected_warehouses)))

# Loop through each selected warehouse and plot the ratios
for n, warehouse in enumerate(selected_warehouses):
    for d in range(7):
        dt = ratio_orders_per_week_warehouse_weekday.loc[ratio_orders_per_week_warehouse_weekday.warehouse == warehouse, ('orders', d)]
        dt = dt[15:-15]  # Exclude the first and last 15 values
        ratio_weekday.loc[d, warehouse] = dt.median()
        ax[n].plot(range(len(dt)), dt, label=f'Day {d}', linewidth=3)
        
    ax[n].set_title(warehouse, fontsize=20)
    ax[n].legend(fontsize=18)

plt.show()

Notes:

* Frankfurt and Munich warehouses do not open on Sundays, that's why we see a constant line on zero.
* Suddently drops to zero means that the warehouse was closed on that day.
* Friday (Day 4) seems to be the best day of the week

In [ ]:
import plotly.express as px

ratio_weekday_mean = ratio_weekday.mean(axis=1)
ratio_weekday['mean'] = ratio_weekday_mean

ratio_weekday.style.bar(subset=['mean'], color=px.colors.qualitative.Set2[7])\
        .background_gradient(subset=['Prague_1'], cmap='BuGn')\
        .background_gradient(subset=['Prague_2'], cmap='BuGn')\
        .background_gradient(subset=['Prague_3'], cmap='BuGn')\
        .background_gradient(subset=['Brno_1'], cmap='BuGn')\
        .background_gradient(subset=['Budapest_1'], cmap='BuGn')\
        .background_gradient(subset=['Frankfurt_1'], cmap='BuGn_r')\
        .background_gradient(subset=['Munich_1'], cmap='BuGn_r')

### 2.3 Seasonal Plots

In [ ]:

def seasonal_plot(X, y, period, freq, ax=None):
    if ax is None:
        _, ax = plt.subplots()
    ax = sns.lineplot(
        x=freq,
        y=y,
        hue=period,
        data=X,
        ci=False,
        ax=ax,
        legend=False,
    )
    ax.set_title(f"Seasonal Plot ({period}/{freq})")
    for line, name in zip(ax.lines, X[period].unique()):
        y_ = line.get_ydata()[-1]
        ax.annotate(
            name,
            xy=(1, y_),
            xytext=(6, 0),
            color=line.get_color(),
            xycoords=ax.get_yaxis_transform(),
            textcoords="offset points",
            size=14,
            va="center",
        )
    return ax

fig, axes = plt.subplots(len(warehouses), 1, figsize=(11, 6 * len(warehouses)), sharex=True)

for warehouse, ax in zip(warehouses, axes):
    warehouse_data = train[train['warehouse'] == warehouse]
    seasonal_plot(warehouse_data, y="orders", period="year", freq="dayofyear", ax=ax)
    ax.set_title(f"Seasonal Plot for {warehouse} (year/dayofyear)")

plt.tight_layout()
plt.show()

Notes:

* Munich and Frankfurt operations are completely different from the other warehouses, and they also show a clearly stronger performance in 2024 compared to previous years.
* Budapest, Brno, and Prague all have their worst month in July.
* All warehouses show both weekly and yearly seasonality.

## 2.4 Autocorrelation Analysis

ACF measures the linear relationship between lagged values of a time series

PACF measures the correlation between lagged values in a time series when we remove the influence of correlated lagged values in between.

In [ ]:

brno_data = train[train['warehouse'] == 'Brno_1']
brno_data = brno_data[brno_data['year'] != 2020]
brno_data.reset_index(drop=True, inplace=True)
brno_data = brno_data[['date', 'orders']]
brno_data.set_index('date', inplace=True)
brno_data = brno_data.asfreq('D')
brno_data.fillna(method='ffill', inplace=True)

pr1_data = train[train['warehouse'] == 'Prague_1']
pr1_data = pr1_data[pr1_data['year'] != 2020]
pr1_data.reset_index(drop=True, inplace=True)
pr1_data = pr1_data[['date', 'orders']]
pr1_data.set_index('date', inplace=True)
pr1_data = pr1_data.asfreq('D')
pr1_data.fillna(method='ffill', inplace=True)

pr2_data = train[train['warehouse'] == 'Prague_2']
pr2_data = pr2_data[pr2_data['year'] != 2020]
pr2_data.reset_index(drop=True, inplace=True)
pr2_data = pr2_data[['date', 'orders']]
pr2_data.set_index('date', inplace=True)
pr2_data = pr2_data.asfreq('D')
pr2_data.fillna(method='ffill', inplace=True)

pr3_data = train[train['warehouse'] == 'Prague_3']
pr3_data = pr3_data[pr3_data['year'] != 2020]
pr3_data.reset_index(drop=True, inplace=True)
pr3_data = pr3_data[['date', 'orders']]
pr3_data.set_index('date', inplace=True)
pr3_data = pr3_data.asfreq('D')
pr3_data.fillna(method='ffill', inplace=True)

bdp_data = train[train['warehouse'] == 'Budapest_1']
bdp_data = bdp_data[bdp_data['year'] != 2020]
bdp_data = bdp_data[bdp_data['year'] != 2021]
bdp_data = bdp_data[bdp_data['date'] != '2024-03-14']
bdp_data.reset_index(drop=True, inplace=True)
bdp_data = bdp_data[['date', 'orders']]
bdp_data.set_index('date', inplace=True)
bdp_data = bdp_data.asfreq('D')
bdp_data.fillna(method='ffill', inplace=True)

munich_data = train[train['warehouse'] == 'Munich_1']
munich_data = munich_data[munich_data['year'] != 2021]
munich_data.reset_index(drop=True, inplace=True)
munich_data = munich_data[['date', 'orders']]
munich_data.set_index('date', inplace=True)
munich_data = munich_data.asfreq('D')
munich_data.fillna(method='ffill', inplace=True)

fkr_data = train[train['warehouse'] == 'Frankfurt_1']
fkr_data = fkr_data[fkr_data['year'] != 2022]
fkr_data.reset_index(drop=True, inplace=True)
fkr_data = fkr_data[['date', 'orders']]
fkr_data.set_index('date', inplace=True)
fkr_data = fkr_data.asfreq('D')
fkr_data.fillna(method='ffill', inplace=True)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
def plot_acf_pacf(data, title):
    fig, axes = plt.subplots(1, 2, figsize=(16, 4))
    plot_acf(data, lags=30, ax=axes[0])
    axes[0].set_title(f'{title} - ACF', fontsize=15)
    plot_pacf(data, lags=30, ax=axes[1])
    axes[1].set_title(f'{title} - PACF', fontsize=15)
    plt.tight_layout()
    plt.show()

def make_stationary(data):
    result = adfuller(data)
    if result[1] > 0.05:
        data = data.diff().dropna() #detrend
    return data

warehouse_data = {
    'Brno_1': brno_data,
    'Prague_1': pr1_data,
    'Prague_2': pr2_data,
    'Prague_3': pr3_data,
    'Budapest_1': bdp_data,
    'Munich_1': munich_data,
    'Frankfurt_1': fkr_data
}


for warehouse, data in warehouse_data.items():
    orders_data = data['orders'].copy()
    stationary_orders_data = make_stationary(orders_data)
    plot_acf_pacf(stationary_orders_data, warehouse)

Notes

* None of the time series are random walk processes, which is expected.
* Weekly seasonality is very strong
* The pattern described by the ACF plots suggest that an autoregressive process is at play
* Negative coefficients and significant autocorrelation at large lags imply that these series are not moving average processes.